In [ ]:
from pyrosm import OSM
from pyrosm import get_data
import pandas as pd
import os
import osmium


In [2]:
district_name = "Somerset"

In [ ]:


# Create map directory if not exists
if not os.path.exists('map'):
    os.makedirs('map')
    print("Created map directory")

print("Downloading Kentucky PBF...")
# Download Kentucky PBF
try:
    fp = get_data("Kentucky", directory="./map")
    print(f"Downloaded to: {fp}")
except Exception as e:
    print(f"Error downloading data: {e}")

Downloaded to: /home/kaveh/projects/osm_to_road_network/map/kentucky-latest.osm.pbf


In [5]:
fp = "map/kentucky-latest.osm.pbf" 
osm = OSM(fp)

def get_boundaries_list(osm):
    boundaries = osm.get_boundaries()
    boundaries_filtered = boundaries[(boundaries["name"].notna()) & (pd.to_numeric(boundaries["admin_level"], errors='coerce').notna())]
    boundaries_filtered["admin_level"] = boundaries_filtered["admin_level"].astype(int)
    boundaries_filtered = boundaries_filtered[boundaries_filtered["admin_level"] == 8]
    return boundaries_filtered.set_index("name")

district_df = get_boundaries_list(osm)
del osm

/home/kaveh/projects/osm_to_road_network/venv/lib/python3.10/site-packages/geopandas/geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [6]:
district_df.to_csv("districts.csv")

In [8]:
district_df_test = pd.read_csv("districts.csv").set_index("name")
district_df_test.head()


,visible,admin_level,boundary,id,timestamp,version,tags,osm_type,geometry,operator,ref,website,border_type,start_date,changeset
name,,,,,,,,,,,,,,,
Annville,False,8,administrative,34170588,1642124238,3,"{""source"":""TIGER/Line\u00AE 2008 Place Shapefi...",way,POLYGON ((-83.98082733154297 37.31906890869140...,NaN,NaN,NaN,NaN,NaN,NaN
Old Shawneetown,NaN,8,administrative,2882184,1735773935,8,"{""source"":""TIGER/Line\u00AE 2008 Place Shapefi...",relation,"POLYGON ((-88.13854217529297 37.7051887512207,...",NaN,NaN,NaN,village,NaN,0.0
Rosiclare,NaN,8,administrative,3252519,1737116750,7,"{""gnis:feature_id"":""2396436"",""source"":""TIGER/L...",relation,POLYGON ((-88.35881805419922 37.42341232299805...,NaN,NaN,NaN,city,NaN,0.0
Elizabethtown,NaN,8,administrative,3378213,1737116750,5,"{""gnis:feature_id"":""2398801"",""source"":""TIGER/L...",relation,POLYGON ((-88.30655670166016 37.44411849975586...,NaN,NaN,NaN,village,NaN,0.0
Cave-In-Rock,NaN,8,administrative,3503908,1735773935,7,"{""gnis:feature_id"":""2397577"",""source"":""TIGER/L...",relation,POLYGON ((-88.17057037353516 37.46570205688476...,NaN,NaN,NaN,village,NaN,0.0


In [9]:
bbox_geom = district_df.loc[district_name]["geometry"]
osm_burnaby = OSM(fp, bounding_box=bbox_geom)
nodes_gdf, edges_gdf = osm_burnaby.get_network(network_type="driving", nodes=True)

In [10]:
import osmnx as ox
import networkx as nx
G = osm_burnaby.to_graph(nodes_gdf, edges_gdf, graph_type="networkx", osmnx_compatible=True)

In [12]:
#graph simplification
G_simplified = ox.simplify_graph(G)

loop_edges = list()
for edge in G_simplified.edges():
    if edge[0] == edge[1]:
        loop_edges.append(edge)
G_simplified.remove_edges_from(loop_edges)

In [13]:
edges_simplified = nx.to_pandas_edgelist(G_simplified, source="source", target="target")
mylist = ["id","source","target","length","maxspeed","geometry"]
edges_simplified

,source,target,u,version,osmid,length,key,v,surface,service,...,oneway,name,motor_vehicle,maxspeed,geometry,timestamp,ref,highway,tags,foot
0,167790704,167862530,167790704,4,16350899,80.287,0.0,167862530,None,None,...,None,Columbia Street,None,None,LINESTRING (-84.60797882080078 37.091793060302...,1360726571,KY 80 Business,secondary,"{""visible"":false,""tiger:cfcc"":""A31"",""tiger:cou...",None
1,167790704,167790717,"[167790704, 167790705, 167790707, 167790708, 1...",8,16339700,102.097,NaN,"[167790705, 167790707, 167790708, 167790710, 1...",None,None,...,None,West Mount Vernon Street,None,None,LINESTRING (-84.60797882080078 37.091793060302...,1733819607,KY 80 Business,secondary,"{""visible"":false,""tiger:cfcc"":""A31"",""tiger:cou...",None
2,167790704,167825885,"[167825888, 167825890, 167825894, 167825897, 1...",5,98082668,323.104,NaN,"[167825888, 167825890, 167825894, 167825897, 1...",None,None,...,None,South Richardson Drive,None,None,LINESTRING (-84.60797882080078 37.091793060302...,1753189703,KY 2303,tertiary,"{""visible"":false,""maxweight"":""44000 lbs"",""sour...",None
3,167790717,167790719,167790717,8,16339700,111.432,0.0,167790719,None,None,...,None,West Mount Vernon Street,None,None,LINESTRING (-84.60697937011719 37.091552734375...,1733819607,KY 80 Business,secondary,"{""visible"":false,""tiger:cfcc"":""A31"",""tiger:cou...",None
4,167790717,167862553,"[167899243, 167790717, 167899246]",4,16344403,84.317,NaN,"[167862553, 167899243, 167899246]",None,None,...,None,Church Street,None,None,LINESTRING (-84.60697937011719 37.091552734375...,1625349597,None,residential,"{""visible"":false,""tiger:cfcc"":""A41"",""tiger:cou...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5895,13116669234,13116669235,13116669234,1,1426878589,18.875,0.0,13116669235,None,driveway,...,None,None,None,None,LINESTRING (-84.6103286743164 37.0544357299804...,1756918096,None,service,"{""visible"":false}",None
5896,13116669234,13116669241,13116669234,1,1426878590,88.712,0.0,13116669241,None,parking_aisle,...,None,None,None,None,LINESTRING (-84.60979461669922 37.055133819580...,1756918096,None,service,"{""visible"":false}",None
5897,13116669241,13116669234,13116669241,1,1426878590,88.712,0.0,13116669234,None,parking_aisle,...,None,None,None,None,LINESTRING (-84.60979461669922 37.055133819580...,1756918096,None,service,"{""visible"":false}",None
5898,13116669241,13116669233,13116669241,1,1426878590,23.801,0.0,13116669233,None,parking_aisle,...,None,None,None,None,LINESTRING (-84.60961151123047 37.055290222167...,1756918096,None,service,"{""visible"":false}",None


In [14]:
import re


def predict_maxspeed(highway: str) -> int:
    """
    Predict a reasonable maxspeed (in km/h) based on the highway type.
    
    Parameters:
    - highway (str): OSM highway tag

    Returns:
    - int: predicted speed in km/h
    """
    if type(highway) == list:
        highway = highway[0]  # take the first type if multiple are present
    highway = highway.lower() if highway else ""

    if highway in ["motorway", "motorway_link"]:
        return 110
    elif highway in ["trunk", "trunk_link"]:
        return 90
    elif highway in ["primary", "primary_link"]:
        return 70
    elif highway in ["secondary", "secondary_link"]:
        return 60
    elif highway in ["tertiary", "tertiary_link"]:
        return 50
    elif highway in ["residential", "living_street"]:
        return 30
    elif highway in ["service"]:
        return 20
    elif highway in ["unclassified", "road"]:
        return 40
    else:
        return 50  # fallback
    
def fix_speed_format(df):
    """
    data = {
    'maxspeed': [
        '25 mph',
        '40 km/h',
        'Unknown',
        '30 mph',
        np.nan,
        '50',  # Assume km/h if no unit is specified
        '60KMH', # Case insensitive for units
        '100 kph',
        '' # Empty string
    ]
    }
    """
    speed_parts = df['maxspeed'].astype(str).str.extract(r'(\d+\.?\d*)\s*(mph|km/h|kmh|kph)?', flags=re.IGNORECASE)

    # Assign extracted parts to temporary columns
    df['speed_value'] = pd.to_numeric(speed_parts[0], errors='coerce')
    df['speed_unit'] = speed_parts[1].str.lower() # Convert units to lowercase for easier comparison

    # Define the conversion factor from miles per hour to kilometers per hour
    MPH_TO_KMPH = 1.60934

    # Initialize the new 'maxspeed_kmph' column with the extracted numeric value
    df['maxspeed'] = df['speed_value']

    # Apply conversion where the unit is 'mph'
    df.loc[df['speed_unit'] == 'mph', 'maxspeed'] = df['speed_value'] * MPH_TO_KMPH

    # Drop the temporary columns if you don't need them
    df = df.drop(columns=['speed_value', 'speed_unit'])
    return df



In [15]:
print("fixing speed format ... ")
edges_simplified = fix_speed_format(edges_simplified)

print(f"predicting missed maxspeed ...")
edges_simplified["maxspeed"] = edges_simplified.apply(lambda row: predict_maxspeed(row.highway) if pd.isna(row.maxspeed) else row.maxspeed, axis=1                             )
edges_simplified['maxspeed'] = edges_simplified['maxspeed'].astype(float)

fixing speed format ... 
predicting missed maxspeed ...


In [16]:
#edges_simplified['id'] = edges_simplified.apply(lambda row: f"{row['source']}_{row['target']}", axis=1)
edges_simplified['id'] = edges_simplified.apply(lambda row: (row['source'], row['target']), axis=1)
mylist = ["id","source","target","length","maxspeed","geometry"]
edges_simplified = edges_simplified[mylist]
edges_simplified

,id,source,target,length,maxspeed,geometry
0,"(167790704, 167862530)",167790704,167862530,80.287,60.0,LINESTRING (-84.60797882080078 37.091793060302...
1,"(167790704, 167790717)",167790704,167790717,102.097,60.0,LINESTRING (-84.60797882080078 37.091793060302...
2,"(167790704, 167825885)",167790704,167825885,323.104,50.0,LINESTRING (-84.60797882080078 37.091793060302...
3,"(167790717, 167790719)",167790717,167790719,111.432,60.0,LINESTRING (-84.60697937011719 37.091552734375...
4,"(167790717, 167862553)",167790717,167862553,84.317,30.0,LINESTRING (-84.60697937011719 37.091552734375...
...,...,...,...,...,...,...
5895,"(13116669234, 13116669235)",13116669234,13116669235,18.875,20.0,LINESTRING (-84.6103286743164 37.0544357299804...
5896,"(13116669234, 13116669241)",13116669234,13116669241,88.712,20.0,LINESTRING (-84.60979461669922 37.055133819580...
5897,"(13116669241, 13116669234)",13116669241,13116669234,88.712,20.0,LINESTRING (-84.60979461669922 37.055133819580...
5898,"(13116669241, 13116669233)",13116669241,13116669233,23.801,20.0,LINESTRING (-84.60961151123047 37.055290222167...


In [17]:
#remove self loops

edges_simplified = edges_simplified[edges_simplified.apply(lambda row: row['source']!=row['target'], axis=1)]


In [18]:
edges_simplified.to_csv(district_name+"_driving_simplified_edges.csv", index=False)

In [24]:
class RestrictionHandler(osmium.SimpleHandler):
    """
    Osmium handler to extract turn restrictions from OSM relations.
    Stores restrictions as dictionaries with 'from', 'via', and 'to' references.
    """
    def __init__(self):
        super().__init__()
        self.restrictions = []

    def relation(self, r):
        if 'restriction' in r.tags:
            rel = {
                "id": r.id,
                "restriction": r.tags["restriction"],
                "from": None,
                "via": None,
                "to": None
            }
            for m in r.members:
                if m.role == "from" and m.type == "w":
                    rel["from"] = str(m.ref)
                elif m.role == "via" and m.type == "n":
                    rel["via"] = str(m.ref)
                elif m.role == "to" and m.type == "w":
                    rel["to"] = str(m.ref)
            self.restrictions.append(rel)


In [26]:
print("Extracting turn restrictions using Osmium...")
handler = RestrictionHandler()
handler.apply_file(fp, locations=False)

Extracting turn restrictions using Osmium...


In [31]:
restriction_df = pd.DataFrame(handler.restrictions)


In [32]:
restriction_df.head()

,id,restriction,from,via,to
0,1566981,only_straight_on,597984891,164583561,111336813
1,1566982,only_straight_on,111336808,164849815,16229343
2,1566983,only_straight_on,111336907,164583561,111336876
3,1630694,no_left_turn,353504923,168212388,118267558
4,1630695,no_left_turn,353504927,168212179,118267555


In [33]:
forbidden = list()
for idx, row in restriction_df.iterrows():
    via_node = row['via']
    from_way = row['from']
    to_way = row['to']
    if via_node in G_simplified.nodes:
        # Find all incoming edges to the 'via' node
        incoming_edges = G_simplified.in_edges(via_node, data=True)
        # Find all outgoing edges from the 'via' node
        outgoing_edges = G_simplified.out_edges(via_node, data=True)
        
        # Identify the specific incoming edge that matches the 'from' way
        from_edge = None
        for u, v, data in incoming_edges:
            if 'osmid' in data and (isinstance(data['osmid'], list) and from_way in data['osmid'] or data['osmid'] == from_way):
                from_edge = (u, v)
                break
        
        # Identify the specific outgoing edge that matches the 'to' way
        to_edge = None
        for u, v, data in outgoing_edges:
            if 'osmid' in data and (isinstance(data['osmid'], list) and to_way in data['osmid'] or data['osmid'] == to_way):
                to_edge = (u, v)
                break
        
        # If both edges are found, add the forbidden turn to the list
        if from_edge and to_edge:
            forbidden.append((from_edge, to_edge))

In [34]:
edge_graph = list()
for node in G_simplified.nodes:
    # Find all incoming edges to the 'via' node
    incoming_edges = G_simplified.in_edges(node, data=False)
    # Find all outgoing edges from the 'via' node
    outgoing_edges = G_simplified.out_edges(node, data=False)
    for u, v in incoming_edges:
        for x, y in outgoing_edges:
            edge_graph.append( ((u, v), (x, y)) )
    
        


In [36]:
new_edge_graph = list(set(edge_graph) - set(forbidden))
len(new_edge_graph)

16301

In [38]:
edge_graph_df = pd.DataFrame(new_edge_graph, columns=['incoming_edge', 'outgoing_edge'])
edge_graph_df = edge_graph_df[edge_graph_df.apply(lambda row: row['incoming_edge'] != row['outgoing_edge'], axis=1) ]
edge_graph_df.to_csv(district_name+"_driving_edge_graph.csv", index=False)

In [42]:
nodes_df = pd.DataFrame(nx.get_node_attributes(G_simplified, 'geometry').items(), columns=['id','geometry'])
nodes_df.to_csv(district_name+"_driving_simplified_nodes.csv", index=False)

In [ ]:
from shapely import wkt
from shapely.geometry import Point
#nodes_df["geometry"] = nodes_df["geometry"].apply(lambda x: wkt.loads(x))

TypeError: Expected bytes or string, got Point

In [45]:
nodes_df.head()

,id,geometry
0,167790704,POINT (-84.60797882080078 37.091793060302734)
1,167790717,POINT (-84.60697937011719 37.091552734375)
2,167790719,POINT (-84.60577392578125 37.091835021972656)
3,167790745,POINT (-84.59418487548828 37.085689544677734)
4,167790747,POINT (-84.59452056884766 37.086551666259766)


In [46]:
nodes_df.set_index("id", inplace=True)
nodes_df.head()

,geometry
id,
167790704,POINT (-84.60797882080078 37.091793060302734)
167790717,POINT (-84.60697937011719 37.091552734375)
167790719,POINT (-84.60577392578125 37.091835021972656)
167790745,POINT (-84.59418487548828 37.085689544677734)
167790747,POINT (-84.59452056884766 37.086551666259766)


In [48]:
edges_df = pd.read_csv(district_name+"_driving_simplified_edges.csv")
edges_df.head()

,id,source,target,length,maxspeed,geometry
0,"(167790704, 167862530)",167790704,167862530,80.287,60.0,LINESTRING (-84.60797882080078 37.091793060302...
1,"(167790704, 167790717)",167790704,167790717,102.097,60.0,LINESTRING (-84.60797882080078 37.091793060302...
2,"(167790704, 167825885)",167790704,167825885,323.104,50.0,LINESTRING (-84.60797882080078 37.091793060302...
3,"(167790717, 167790719)",167790717,167790719,111.432,60.0,LINESTRING (-84.60697937011719 37.091552734375...
4,"(167790717, 167862553)",167790717,167862553,84.317,30.0,LINESTRING (-84.60697937011719 37.091552734375...


In [52]:
import h3
edges_df["incoming_cell"] = edges_df["target"].apply(lambda t: h3.latlng_to_cell(nodes_df.loc[t]["geometry"].y, nodes_df.loc[t]["geometry"].x, 15))
edges_df["outgoing_cell"] = edges_df["source"].apply(lambda t: h3.latlng_to_cell(nodes_df.loc[t]["geometry"].y, nodes_df.loc[t]["geometry"].x, 15))

In [53]:
def find_lca(cell1: str, cell2: str) -> str:
    """
    Find the Lowest Common Ancestor (LCA) cell between two H3 cells.
    
    The LCA is the coarsest resolution cell that contains both input cells.
    
    Args:
        cell1: First H3 cell ID
        cell2: Second H3 cell ID
    
    Returns:
        LCA cell ID, or None if no common ancestor exists
    """
    if cell1 is None or cell2 is None:
        return None
    
    cell1_res = h3.get_resolution(cell1)
    cell2_res = h3.get_resolution(cell2)
    lca_res = min(cell1_res, cell2_res)
    
    while lca_res >= 0:
        if h3.cell_to_parent(cell1, lca_res) == h3.cell_to_parent(cell2, lca_res):
            return h3.cell_to_parent(cell1, lca_res)
        lca_res -= 1
    
    return None


def find_resolution(cell: str) -> int:
    """
    Extract H3 resolution from a cell ID.
    
    Args:
        cell: H3 cell ID
    
    Returns:
        Resolution level (0-15), or -1 if cell is None
    """
    if cell is None:
        return -1
    return h3.get_resolution(cell)

def LCA_Resolution(cell1: str, cell2: str) -> int:
    """
    Find the resolution of the Lowest Common Ancestor (LCA) cell between two H3 cells.
    
    Args:
        cell1: First H3 cell ID
        cell2: Second H3 cell ID
    
    Returns:
        Resolution level (0-15), or -1 if no common ancestor exists
    """
    lca = find_lca(cell1, cell2)
    if lca is None:
        return -1
    return find_resolution(lca) 

In [54]:
edges_df["lca_res"] = edges_df.apply(lambda row: LCA_Resolution(row["incoming_cell"], row["outgoing_cell"]), axis=1)
edges_df.head()

,id,source,target,length,maxspeed,geometry,incoming_cell,outgoing_cell,lca_res
0,"(167790704, 167862530)",167790704,167862530,80.287,60.0,LINESTRING (-84.60797882080078 37.091793060302...,8f44cb2c92f38a0,8f44cb2c928db05,8
1,"(167790704, 167790717)",167790704,167790717,102.097,60.0,LINESTRING (-84.60797882080078 37.091793060302...,8f44cb2c928e8c9,8f44cb2c928db05,10
2,"(167790704, 167825885)",167790704,167825885,323.104,50.0,LINESTRING (-84.60797882080078 37.091793060302...,8f44cb2c938d798,8f44cb2c928db05,8
3,"(167790717, 167790719)",167790717,167790719,111.432,60.0,LINESTRING (-84.60697937011719 37.091552734375...,8f44cb2c929809b,8f44cb2c928e8c9,9
4,"(167790717, 167862553)",167790717,167862553,84.317,30.0,LINESTRING (-84.60697937011719 37.091552734375...,8f44cb2c928b12d,8f44cb2c928e8c9,10


In [55]:
edges_df.to_csv(district_name+"_driving_simplified_edges_with_h3.csv", index=False)

In [56]:
edge_graph = pd.read_csv(district_name+"_driving_edge_graph.csv")


In [57]:
# next_edge in this point is exactly outgoing_edge, but for shortcut edges it can be different
# cost of two edges path is equal of cost of first edge
# cost of shortcut edges is sum of costs of underlying edges minus the cost of the last edge
# for calculate final cost of path, we add cost of last edge separately

edge_graph["next_edge"] = edge_graph["outgoing_edge"]
def dummy_cost(length, maxspeed):   
    #calculete travel time for the cost
    # in edge_df we have length in meters and maxspeed in km/h
    travel_time = length / (maxspeed * 1000 / 3600)  # convert km/h to m/s and calculate time in seconds
    return travel_time

edges_df["cost"] = edges_df.apply(lambda row: dummy_cost(row["length"], row["maxspeed"]), axis=1)
edges_df.head()

,id,source,target,length,maxspeed,geometry,incoming_cell,outgoing_cell,lca_res,cost
0,"(167790704, 167862530)",167790704,167862530,80.287,60.0,LINESTRING (-84.60797882080078 37.091793060302...,8f44cb2c92f38a0,8f44cb2c928db05,8,4.817220
1,"(167790704, 167790717)",167790704,167790717,102.097,60.0,LINESTRING (-84.60797882080078 37.091793060302...,8f44cb2c928e8c9,8f44cb2c928db05,10,6.125820
2,"(167790704, 167825885)",167790704,167825885,323.104,50.0,LINESTRING (-84.60797882080078 37.091793060302...,8f44cb2c938d798,8f44cb2c928db05,8,23.263488
3,"(167790717, 167790719)",167790717,167790719,111.432,60.0,LINESTRING (-84.60697937011719 37.091552734375...,8f44cb2c929809b,8f44cb2c928e8c9,9,6.685920
4,"(167790717, 167862553)",167790717,167862553,84.317,30.0,LINESTRING (-84.60697937011719 37.091552734375...,8f44cb2c928b12d,8f44cb2c928e8c9,10,10.118040


In [58]:
edges_df.set_index(["source","target"], inplace=True)

In [59]:
edges_df.reset_index(inplace=True)
edges_df.set_index("id", inplace=True)

In [60]:
shortcut_table = edge_graph.copy()
shortcut_table["cost"] = shortcut_table.apply(lambda row: edges_df.loc[row["incoming_edge"]]["cost"], axis=1)
shortcut_table["via_cell"] = shortcut_table.apply(lambda row: edges_df.loc[row["incoming_edge"]]["incoming_cell"], axis=1)
shortcut_table["via_cell_res"] = 15

shortcut_table["lca_res_incoming_edge"] = shortcut_table["incoming_edge"].apply(lambda x: edges_df.loc[x]["lca_res"])
shortcut_table["lca_res_outgoing_edge"] = shortcut_table["outgoing_edge"].apply(lambda x: edges_df.loc[x]["lca_res"])
shortcut_table["lca_res"] = shortcut_table.apply(lambda row: max(row["lca_res_incoming_edge"], row["lca_res_outgoing_edge"]), axis=1)


shortcut_table.head()

,incoming_edge,outgoing_edge,next_edge,cost,via_cell,via_cell_res,lca_res_incoming_edge,lca_res_outgoing_edge,lca_res
0,"(7016006731, 7016006763)","(7016006763, 7016006760)","(7016006763, 7016006760)",2.47050,8f2669a4d248da2,15,11,-1,11
1,"(167873255, 11980662584)","(11980662584, 167845227)","(11980662584, 167845227)",14.13504,8f2669a6ba56d00,15,8,11,11
2,"(11980909215, 11980909197)","(11980909197, 11980909196)","(11980909197, 11980909196)",3.76776,8f2669a69d23c5e,15,10,10,10
3,"(167862553, 167790717)","(167790717, 167790704)","(167790717, 167790704)",10.11804,8f44cb2c928e8c9,15,10,10,10
4,"(167862622, 167978652)","(167978652, 167848877)","(167978652, 167848877)",22.19076,8f2669a6b29c2a3,15,9,7,9
